In [1]:
import pandas as pd

In [2]:
#1. Load the Dataset
file_path = '/content/chipotle.xlsx'
df = pd.read_excel(file_path)

In [6]:
#2. Check for any missing values
missing_values = df.isnull().sum()
print("Missing values in each column: ", missing_values)

Missing values in each column:  order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64


In [12]:
#For 'Choice Description', use "No Description" to indicate a missing description
df['choice_description'].fillna("No Description", inplace=True)
df.isnull().sum()

<ipython-input-12-d7cfd667e7bf>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['choice_description'].fillna("No Description", inplace=True)


,0
order_id,0
quantity,0
item_name,0
choice_description,0
item_price,0


In [14]:
#3. Verify the Data Types
print("\nData types before conversion:\n", df.dtypes)
# Assuming we expect 'Order ID' and 'Quantity' as integers, 'Item Price' as float
df['order_id'] = df['order_id'].astype('Int64', errors='ignore')
df['quantity'] = df['quantity'].astype('Int64', errors='ignore')
df['item_price'] = df['item_price'].replace('[\$,]', '', regex=True).astype(float)  # Removes $ if present

print("\nData types after conversion:\n", df.dtypes)


Data types before conversion:
 order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

Data types after conversion:
 order_id                Int64
quantity                Int64
item_name              object
choice_description     object
item_price            float64
dtype: object


In [15]:
#4. Handle duplicated entries
duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate entries: {duplicates}")
df = df.drop_duplicates()


Number of duplicate entries: 59


In [16]:
#5. Analyze Quantity and Item Price for Anomalies
# Checking for any zero or negative values in Quantity and Item Price
quantity_anomalies = df[df['quantity'] <= 0]
price_anomalies = df[df['item_price'] <= 0]
print(f"\nQuantity anomalies:\n{quantity_anomalies}")
print(f"\nPrice anomalies:\n{price_anomalies}")


Quantity anomalies:
Empty DataFrame
Columns: [order_id, quantity, item_name, choice_description, item_price]
Index: []

Price anomalies:
Empty DataFrame
Columns: [order_id, quantity, item_name, choice_description, item_price]
Index: []


In [17]:
#6. Examine Choice Descriptions
# Checking for consistency in 'Choice Description'
choice_desc_counts = df['choice_description'].value_counts()
print("\nChoice Description frequency:\n", choice_desc_counts)


Choice Description frequency:
 choice_description
No Description                                                                                                                                   1228
[Diet Coke]                                                                                                                                       133
[Coke]                                                                                                                                            115
[Sprite]                                                                                                                                           77
[Fresh Tomato Salsa, [Rice, Black Beans, Cheese, Sour Cream, Lettuce]]                                                                             41
                                                                                                                                                 ... 
[Tomatillo Green Chili Salsa, [Sour Cream, Cheese

In [20]:
#6. Check for Special Characters
# Looking for special characters in 'Item Name' and 'Choice Description'
# Removing special characters from 'Item Name' and 'Choice Description' columns using .loc to avoid warnings
df.loc[:, 'item_name'] = df['item_name'].str.replace(r'[^\w\s]', '', regex=True)
df.loc[:, 'choice_description'] = df['choice_description'].str.replace(r'[^\w\s]', '', regex=True)
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,No Description,2.39
1,1,1,Izze,Clementine,3.39
2,1,1,Nantucket Nectar,Apple,3.39
3,1,1,Chips and TomatilloGreen Chili Salsa,No Description,2.39
4,2,2,Chicken Bowl,TomatilloRed Chili Salsa Hot Black Beans Rice ...,16.98


In [22]:
#7. Validate Order ID Integrity
# Ensuring no duplicates in 'Order ID' for unique orders
order_id_duplicates = df['order_id'].duplicated().sum()
print(f"\nNumber of duplicate Order IDs: {order_id_duplicates}")
unique_orderids = df['order_id'].drop_duplicates().shape[0]
print(f"\nNumber of unique Order IDs: {unique_orderids}")


Number of duplicate Order IDs: 2729

Number of unique Order IDs: 1834


In [24]:
#8. Standardize Item Names
df.loc[:, 'item_name'] = df['item_name'].str.strip().str.lower()
df.head()

,order_id,quantity,item_name,choice_description,item_price
0,1,1,chips and fresh tomato salsa,No Description,2.39
1,1,1,izze,Clementine,3.39
2,1,1,nantucket nectar,Apple,3.39
3,1,1,chips and tomatillogreen chili salsa,No Description,2.39
4,2,2,chicken bowl,TomatilloRed Chili Salsa Hot Black Beans Rice ...,16.98


In [26]:
#9. Investigate Quantity-Price Relationship
df.loc[:, 'total_price'] = df['quantity'] * df['item_price']

# Display the first few rows to confirm the new column is added
print(df[['quantity', 'item_price', 'total_price']].head())

#Check for inconsistencies where Total Price does not align with Quantity * Item Price
inconsistent_total_price = df[df['total_price'] != df['quantity'] * df['item_price']]

# Print inconsistent rows, if any
if not inconsistent_total_price.empty:
    print("\nInconsistencies in Total Price calculation:")
    print(inconsistent_total_price[['quantity', 'item_price', 'total_price']])
else:
    print("\nNo inconsistencies found in Total Price calculation.")


   quantity  item_price  total_price
0         1        2.39         2.39
1         1        3.39         3.39
2         1        3.39         3.39
3         1        2.39         2.39
4         2       16.98        33.96

No inconsistencies found in Total Price calculation.


In [29]:
#10. Encode Categorical Data
# For example, encoding 'Item Name' using Label Encoding or One-Hot Encoding
# Using Label Encoding
df.loc[:, 'Item Name Encoded'] = df['item_name'].astype('category').cat.codes

In [30]:
#11. Ensure Consistent Units
#This check ensures all values in `Quantity` are integers and non-negative
quantity_inconsistent = df[df['quantity'] <= 0]
if not quantity_inconsistent.empty:
    print("\nInconsistent units found in Quantity column:")
    print(quantity_inconsistent[['order_ID', 'quantity', 'item_name']])
else:
    print("\nAll entries in Quantity are consistent and non-negative.")

# 2. Check if Item Price is consistent in format and units (e.g., no extreme values, currency symbols)
#    Confirming all prices are within a reasonable range (e.g., no items costing $0 or unrealistic values)
price_inconsistent = df[(df['item_price'] <= 0) | (df['item_price'] > 100)]  # Assuming an upper limit for sanity check
if not price_inconsistent.empty:
    print("\nInconsistent units found in Item Price column:")
    print(price_inconsistent[['order_ID', 'quantity', 'item_name', 'item_name']])
else:
    print("\nAll entries in Item Price are within expected range.")

# Optional: Display unique values in Quantity to verify all values are whole numbers
print("\nUnique values in Quantity column:\n", df['quantity'].unique())

# Optional: Display summary statistics for Item Price to check for any outliers
print("\nSummary statistics for Item Price column:\n", df['item_price'].describe())



All entries in Quantity are consistent and non-negative.

All entries in Item Price are within expected range.

Unique values in Quantity column:
 <IntegerArray>
[1, 2, 3, 4, 5, 15, 7, 8, 10]
Length: 9, dtype: Int64

Summary statistics for Item Price column:
 count    4563.000000
mean        7.490083
std         4.244155
min         1.090000
25%         3.750000
50%         8.750000
75%         9.250000
max        44.250000
Name: item_price, dtype: float64


In [32]:
#Cleaned Data Summary
print("\nCleaned Data Summary:")
print(df.info())

df.head()


Cleaned Data Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 4563 entries, 0 to 4621
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   order_id            4563 non-null   Int64  
 1   quantity            4563 non-null   Int64  
 2   item_name           4563 non-null   object 
 3   choice_description  4563 non-null   object 
 4   item_price          4563 non-null   float64
 5   total_price         4563 non-null   Float64
 6   Item Name Encoded   4563 non-null   int8   
dtypes: Float64(1), Int64(2), float64(1), int8(1), object(2)
memory usage: 267.4+ KB
None


,order_id,quantity,item_name,choice_description,item_price,total_price,Item Name Encoded
0,1,1,chips and fresh tomato salsa,No Description,2.39,2.39,24
1,1,1,izze,Clementine,3.39,3.39,34
2,1,1,nantucket nectar,Apple,3.39,3.39,35
3,1,1,chips and tomatillogreen chili salsa,No Description,2.39,2.39,31
4,2,2,chicken bowl,TomatilloRed Chili Salsa Hot Black Beans Rice ...,16.98,33.96,17


In [34]:
#Conversion of Excel to CSV
df.to_csv('/content/cleaned_chitpole_data.csv', index=False)